In [1]:
import torch.nn as nn
from torchvision import transforms
from mydataset import MyDataset
from torch.utils.data import  DataLoader
import torch
from myunet import Unet
from tqdm import tqdm
from torch.cuda.amp import autocast

In [4]:
batch_size = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
train_set = MyDataset(root="data", is_train=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
]))

val_set = MyDataset(root="data", is_train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
]))

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)
len(train_set), len(val_set), len(train_loader), len(val_loader)

(3800, 200, 38, 2)

In [6]:
model = Unet(6, 2)
model.cuda()

criterion = nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode='max', factor=0.5, patience=2,
    verbose=True)
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [7]:
def train_model(epoch):
    model.train()
    print(f"Epoch {epoch} Training")
    with tqdm(train_loader, desc=str(epoch)) as it:
        for idx, (img1, img2, mask) in enumerate(it, 0):
            img1, img2, mask = img1.cuda(), img2.cuda(), mask.cuda()
            mask = mask.long()
            optim.zero_grad()
            with autocast():
                outputs = model(img1, img2)
                loss = criterion(outputs, mask)
            # loss.backward()
            # optim.step()
            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()
            _, pred = torch.max(outputs.data, 1)
            p, r, f1, iou = get_index(pred, mask)
            it.set_postfix_str(f"loss: {loss.item(): .4f} p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")
            # if idx % 50 == 49:

In [8]:
def get_index(pred, label):
    eps = 1e-7
    tp = torch.sum(label * pred)
    fp = torch.sum(pred) - tp
    fn = torch.sum(label) - tp

    p = (tp + eps) / (tp + fp + eps)
    r = (tp + eps) / (tp + fn + eps)
    f1 = (2 * p * r + eps) / (p + r + eps)
    iou = (tp + eps) / (tp + fn + fp + eps)
    return p, r, f1, iou


def test_model(epoch):
    model.eval()
    global max_score
    f1s = 0
    print(f"Epoch {epoch} Testing")
    with torch.no_grad():
        with tqdm(val_loader, desc=str(epoch)) as it:
            for img1, img2, mask in it:
                img1, img2, mask = img1.cuda(), img2.cuda(), mask.cuda()
                outputs = model(img1, img2)
                _, pred = torch.max(outputs.data, 1)
                p, r, f1, iou = get_index(pred, mask)
                f1s += f1
                it.set_postfix_str(f"p: {p: .4f}  r: {r: .4f}  f1: {f1: .4f}  iou: {iou: .4f}")
    f1s /= len(val_loader)
    scheduler.step(f1s)
    print("f1", f1s.item())
    if max_score < f1s:
        max_score = f1s
        print('max_score', max_score.item())

In [9]:
num_epoch = 10
max_score = 0
for epoch in range(0, num_epoch):
    train_model(epoch=epoch)
    test_model(epoch=epoch)
print("completed!")
print('max_score', max_score.item())

Epoch 0 Training


0: 100%|██████████| 38/38 [00:30<00:00,  1.23it/s, loss:  0.5027 p:  0.7639  r:  0.1845  f1:  0.2973  iou:  0.1746]


Epoch 0 Testing


0: 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, p:  0.3440  r:  0.9259  f1:  0.5017  iou:  0.3348]


f1 0.5137138375704662
max_score 0.5137138375704662
Epoch 1 Training


...

f1 0.6389766343022689
Epoch 9 Training


9: 100%|██████████| 38/38 [00:30<00:00,  1.25it/s, loss:  0.4549 p:  0.6705  r:  0.5752  f1:  0.6192  iou:  0.4484]


Epoch 9 Testing


9: 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, p:  0.7276  r:  0.5020  f1:  0.5941  iou:  0.4226]

f1 0.6256118832324173
completed!
max_score 0.6602825638084134
